In [1]:
import cv2
import numpy as np


In [24]:
def get_harris_corners(input_img, alpha, window_size, threshold , local_maxima=True):
    
    corners = []
    output_img = cv2.cvtColor(input_img.copy(), cv2.COLOR_GRAY2RGB)
    r_values = np.zeros(input_img.shape , dtype=float)
    offset = int(window_size/2)
    
    # Calculate image gradient
    dy, dx = np.gradient(input_img)
    Ixx = dx**2
    Ixy = dy*dx
    Iyy = dy**2
    
#     local_neighborhood_size = 6
    
    for y in range(offset, input_img.shape[0] - offset):
        for x in range(offset, input_img.shape[1] - offset):
            
            # Values of sliding window
            start_y = y - offset
            end_y = y + offset + 1
            start_x = x - offset
            end_x = x + offset + 1
            

            windowIxx = Ixx[start_y : end_y, start_x : end_x]
            windowIxy = Ixy[start_y : end_y, start_x : end_x]
            windowIyy = Iyy[start_y : end_y, start_x : end_x]
            
            # Sum of squares of intensities of partial derevatives 
            Sxx = windowIxx.sum()
            Sxy = windowIxy.sum()
            Syy = windowIyy.sum()

            # Calculate determinant and trace of the matrix
            det = (Sxx * Syy) - (Sxy**2)
            trace = Sxx + Syy
            
            # Calculate r for Harris Corner equation
            r = det - alpha*(trace**2)

            if r > threshold:

                
                r_values[y,x] = r
                if(not local_maxima):
                    corners.append([x, y, r])
                    output_img[y,x] = (0,0,255)
    if(local_maxima):
        local_window_size = 4
        for y in range(local_window_size, input_img.shape[0] - local_window_size):
            for x in range(local_window_size, input_img.shape[1] - local_window_size):
                local_window =  r_values[y-local_window_size:y+local_window_size,x-local_window_size:x+local_window_size]
                if((r_values[y,x] >=local_window.max()) and r_values[y,x] != 0):
                    corners.append([x, y, r_values[y,x]])
                    output_img[y-2:y+2,x-2:x+2] = (0,0,255)



    return corners, output_img



In [25]:
alpha = 0.04
window_size = 5
threshold = 100000000.00

input_img = cv2.imread('../Images/7/Building.jpg', 0)
harris_corner_list, corner_img  = get_harris_corners(input_img, alpha, window_size, threshold , True)
print(len(harris_corner_list))


# cv2.imshow('Detected Corners', corner_img)
# cv2.waitKey(0)


cv2.imwrite("corners_img_local.png", corner_img)


3622


True

In [16]:
alpha = 0.04
window_size = 5
threshold = 100000000.00

input_img = cv2.imread('../Images/7/Building.jpg', 0)
harris_corner_list, corner_img  = get_harris_corners(input_img, alpha, window_size, threshold , False)
print(len(harris_corner_list))


# cv2.imshow('Detected Corners', corner_img)
# cv2.waitKey(0)


cv2.imwrite("corners_img.png", corner_img)

206670


True

In [ ]:
corner_np_array = np.float32(harris_corner_list)

In [12]:
print(input_img.shape)

(1384, 2000)


In [18]:
neighborhood_size = 5
sub_shape = (neighborhood_size, neighborhood_size)
view_shape = tuple(np.subtract(r_values.shape, sub_shape) + 1) + sub_shape
strides = r_values.strides + r_values.strides

sub_matrices = np.lib.stride_tricks.as_strided(r_values, view_shape, strides)
maximas = np.max(sub_matrices, axis=(2, 3))
print(maximas[0:10,0:10])
print(maximas.shape)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.36003706e+07 1.36003706e+07 1.36003706e+07 1.13312441e+07
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.05153417e+07 2.05153417e+07 2.05153417e+07 1.71526911e+07
  1.43482801e+07 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.08917750e+07 3.08917750e+07 3.08917750e+07 2.94396462e+07
  2.81604624e+07 1.63161516e+07 1.31095231e+07 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.10778632e+07 3.10778632e+07 3.10778632e+07 2.98450520e